In [ ]:
!pip install wrds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.1 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.21
    Uninstalling SQLAlchemy-2.0.21:
      Successfully uninstalled SQLAlchemy-2.0.21
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython-sql 0.5.0 requires sqlalchemy>=2.0, but you have sqlalchemy 1.4.49 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
import wrds
import time
from tqdm import tqdm
import cupy as cp
from datetime import timedelta

# Define paths for temporary and final files
temp_crsp_path = 'temp_crsp.csv'
temp_compustat_path = 'temp_compustat.csv'
temp_ibes_path = 'temp_ibes.csv'
temp_linking_table_path = 'temp_linking_table.csv'
final_data_path = 'final_merged_data.csv'
temp_market_return_path = 'temp_market_return.csv'

# Connect to WRDS database
db = wrds.Connection(wrds_username='ziluyang', wrds_password='8KGyqRs6b9$jmsx')

# Get start and end dates for the query from the user
start_date = input("Please enter the start date for the query (format: YYYY-MM-DD): ")
end_date = input("Please enter the end date for the query (format: YYYY-MM-DD): ")

# User input for the number of days before and after the announcement for ABR calculation
days_before = int(input("Enter the number of days before the announcement: "))
days_after = int(input("Enter the number of days after the announcement: "))

# User input for selection
print("Please select the sorting method:")
print("1: Based on six-month return")
print("2: Based on Standardized Unexpected Earnings (SUE)")
print("3: Based on Abnormal Return Around Earnings Announcement")
print("4: Based on Revision in Analyst Forecasts")
print("5: Output all")
choice = input("Enter your choice (1/2/3/4/5): ")

# Retrieve CRSP data
query_crsp = f"""
    SELECT a.permno, a.date, a.ret
    FROM crsp.msf AS a
    JOIN crsp.msenames AS b ON a.permno = b.permno
    WHERE a.date BETWEEN '{start_date}' AND '{end_date}'
    AND b.exchcd BETWEEN 1 AND 3
"""
data_crsp = db.raw_sql(query_crsp)
data_crsp.to_csv(temp_crsp_path, index=False)

# Retrieve Compustat data
query_compustat = f"""
    SELECT gvkey, datadate, ni, prcc_f * csho as mv, seq as bv, oancf as cf
    FROM comp.funda
    WHERE indfmt = 'INDL'
    AND datafmt = 'STD'
    AND popsrc = 'D'
    AND consol = 'C'
    AND datadate BETWEEN '{start_date}' AND '{end_date}'
"""
data_compustat = db.raw_sql(query_compustat)
data_compustat.to_csv(temp_compustat_path, index=False)

# Retrieve I/B/E/S data
query_ibes = f"""
    SELECT ticker, statpers, meanest
    FROM ibes.statsum_epsus
    WHERE statpers BETWEEN '{start_date}' AND '{end_date}'
"""
data_ibes = db.raw_sql(query_ibes)
data_ibes['revision'] = data_ibes.groupby('ticker')['meanest'].diff()  # Calculate revisions
data_ibes.to_csv(temp_ibes_path, index=False)

# Retrieve linking table
query_linking_table = """
    SELECT a.gvkey, b.lpermno as permno
    FROM comp.names AS a
    JOIN crsp.ccmxpf_lnkhist AS b ON a.gvkey = b.gvkey
    WHERE b.linktype IN ('LC', 'LU')
    AND b.linkprim IN ('P', 'C')
"""
linking_table = db.raw_sql(query_linking_table)
linking_table.to_csv(temp_linking_table_path, index=False)

# Query market return data
query_market_return = f"""
    SELECT date, ret
    FROM crsp.msf
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""
data_market_return = db.raw_sql(query_market_return)
data_market_return.to_csv(temp_market_return_path, index=False)

# Query RDQ data
query_rdq = f"""
    SELECT gvkey, rdq
    FROM comp.fundq
    WHERE rdq BETWEEN '{start_date}' AND '{end_date}'
"""
data_rdq = db.raw_sql(query_rdq)
data_rdq.to_csv('temp_rdq.csv', index=False)

# Query market equal-weighted return data
query_market_equal_weighted_return = f"""
    SELECT date, ewretd as market_ret
    FROM crsp.msi
    WHERE date BETWEEN '{start_date}' AND '{end_date}'
"""
data_market_equal_weighted_return = db.raw_sql(query_market_equal_weighted_return)
data_market_equal_weighted_return.to_csv('temp_market_equal_weighted_return.csv', index=False)

# Initialize parameters
chunk_size = 50  # Adjust chunk_size to fit available memory
total_chunks = sum(1 for _ in pd.read_csv(temp_crsp_path, chunksize=chunk_size))
first_chunk = True

with tqdm(total=total_chunks, desc="Processing", unit="chunk") as pbar:
    start_time = time.time()

    # Gradually read and merge data
    for crsp_chunk in pd.read_csv(temp_crsp_path, chunksize=chunk_size):

        crsp_chunk = pd.merge(crsp_chunk, pd.read_csv(temp_linking_table_path), how='inner', on='permno')
        crsp_chunk = pd.merge(crsp_chunk, pd.read_csv(temp_compustat_path), how='inner', left_on='gvkey', right_on='gvkey')

        # Merge RDQ into the current chunk
        data_rdq = pd.read_csv('temp_rdq.csv')
        data_rdq['gvkey'] = data_rdq['gvkey'].astype(int)
        crsp_chunk = pd.merge(crsp_chunk, data_rdq, how='left', on='gvkey')

        # Calculate ABR
        data_market_equal_weighted_return = pd.read_csv('temp_market_equal_weighted_return.csv')
        crsp_chunk = pd.merge(crsp_chunk, data_market_equal_weighted_return, how='left', left_on='date', right_on='date')
        ret_gpu = cp.array(crsp_chunk['ret'].values)
        market_ret_gpu = cp.array(crsp_chunk['market_ret'].values)
        crsp_chunk['abr'] = cp.asnumpy(ret_gpu - market_ret_gpu)

        # For each stock and each announcement date, calculate the average ABR
        for gvkey, group in crsp_chunk.groupby('gvkey'):
            announcement_dates = group['rdq'].dropna().unique()
            for i, announcement_date in enumerate(announcement_dates, 1):
                group['date'] = pd.to_datetime(group['date'])
                announcement_date = pd.to_datetime(announcement_date)
                mask = (group['date'] >= announcement_date - pd.Timedelta(days=days_before)) & (group['date'] <= announcement_date + pd.Timedelta(days=days_after))
                abr_gpu = cp.array(group.loc[mask, 'abr'].values)
                mean_abr = cp.asnumpy(abr_gpu.mean())
                crsp_chunk.loc[group.index, f'abr_{i}'] = mean_abr

        # Data transformation
        crsp_chunk['permno'] = crsp_chunk['permno'].astype(str)
        crsp_chunk = pd.merge(crsp_chunk, pd.read_csv(temp_ibes_path), how='left', left_on='permno', right_on='ticker')

        # Calculate six-month return using GPU
        six_month_return_gpu = cp.empty_like(ret_gpu)
        window_size = 6
        for i in range(len(ret_gpu)):
            start = max(i - window_size + 1, 0)
            six_month_return_gpu[i] = ret_gpu[start:i+1].sum()
        crsp_chunk['six_month_return'] = cp.asnumpy(six_month_return_gpu)

        # Calculate future one, two, three years and six months return using GPU
        for i in range(1, 4):
            future_return_column_name = f'return_{i}y_after'
            months = i * 12
            future_return_gpu = cp.empty_like(ret_gpu)
            for j in range(len(ret_gpu)):
                start = j + 1
                end = start + months
                future_return_gpu[j] = ret_gpu[start:end].sum()
            crsp_chunk[future_return_column_name] = cp.asnumpy(future_return_gpu)

        # Specially add a column for future six months
        future_six_month_return_gpu = cp.empty_like(ret_gpu)
        for i in range(len(ret_gpu)):
            start = i + 1
            end = start + 6
            future_six_month_return_gpu[i] = ret_gpu[start:end].sum()
        crsp_chunk['future_six_month_return'] = cp.asnumpy(future_six_month_return_gpu)

        # Calculate Book-to-Market Ratio and Cash flow-to-price Ratio using GPU
        bv_gpu = cp.array(crsp_chunk['bv'].values)
        mv_gpu = cp.array(crsp_chunk['mv'].values)
        cf_gpu = cp.array(crsp_chunk['cf'].values)

        crsp_chunk['book_to_market'] = cp.asnumpy(bv_gpu / mv_gpu)
        crsp_chunk['cash_flow_to_price'] = cp.asnumpy(cf_gpu / mv_gpu)

        # Calculate SUE
        # Calculate the difference in earnings per share for each quarter
        crsp_chunk['earnings_diff'] = crsp_chunk['ni'] - crsp_chunk['ni'].shift(4)

        # Calculate the standard deviation of the difference in earnings per share for the past eight quarters
        crsp_chunk['F_it'] = crsp_chunk['earnings_diff'].rolling(window=8).std()

        # Calculate SUE using the above model
        crsp_chunk['sue'] = crsp_chunk['earnings_diff'] / crsp_chunk['F_it']

        # Calculate the average revision for the next 6 months
        crsp_chunk['avg_revision_next_6_months'] = crsp_chunk.groupby('permno')['revision'].rolling(window=6).mean().reset_index(level=0, drop=True).shift(-6)

        # Calculate the average revision starting from month 7
        crsp_chunk['avg_revision_from_month_7'] = crsp_chunk.groupby('permno')['revision'].expanding(min_periods=7).mean().reset_index(level=0, drop=True).shift(-7)

        # Print columns of crsp_chunk here
        print(crsp_chunk.columns)

        # Append data chunk to final file
        mode = 'a' if not first_chunk else 'w'
        header = first_chunk
        crsp_chunk.to_csv(final_data_path, mode=mode, header=header, index=False)
        first_chunk = False

        # Update progress bar
        pbar.update(1)

        # Calculate and display remaining time
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / pbar.n) * pbar.total
        remaining_time = estimated_total_time - elapsed_time
        remaining_time_str = str(timedelta(seconds=remaining_time))
        pbar.set_postfix(remaining_time=remaining_time_str)

# Print header information of the final file
print(pd.read_csv(final_data_path, nrows=5))


Loading library list...
Done
请输入查询的开始时间（格式：YYYY-MM-DD）：2000-01-01
请输入查询的截止时间（格式：YYYY-MM-DD）：2002-01-01
请输入公告前的天数：3
请输入公告后的天数：1
请选择排序方式：
1: 根据六个月的收益率
2: 根据Standardized Unexpected Earnings (SUE)
3: 根据Abnormal Return Around Earnings Announcement
4: 根据Revision in Analyst Forecasts
5: 输出全部
请输入您的选择 (1/2/3/4/5): 4


Processing:   0%|          | 1/20015 [00:00<3:37:52,  1.53chunk/s, remaining_time=3:38:13.160342]

Index(['permno', 'date', 'ret', 'gvkey', 'datadate', 'ni', 'mv', 'bv', 'cf',
       'rdq', 'market_ret', 'abr', 'abr_1', 'abr_2', 'abr_3', 'abr_4', 'abr_5',
       'abr_6', 'abr_7', 'abr_8', 'ticker', 'statpers', 'meanest', 'revision',
       'six_month_return', 'return_1y_after', 'return_2y_after',
       'return_3y_after', 'future_six_month_return', 'book_to_market',
       'cash_flow_to_price', 'earnings_diff', 'F_it', 'sue',
       'avg_revision_next_6_months', 'avg_revision_from_month_7'],
      dtype='object')


Processing:   0%|          | 2/20015 [00:01<3:43:20,  1.49chunk/s, remaining_time=3:42:43.248951]

Index(['permno', 'date', 'ret', 'gvkey', 'datadate', 'ni', 'mv', 'bv', 'cf',
       'rdq', 'market_ret', 'abr', 'abr_1', 'abr_2', 'abr_3', 'abr_4', 'abr_5',
       'abr_6', 'abr_7', 'abr_8', 'ticker', 'statpers', 'meanest', 'revision',
       'six_month_return', 'return_1y_after', 'return_2y_after',
       'return_3y_after', 'future_six_month_return', 'book_to_market',
       'cash_flow_to_price', 'earnings_diff', 'F_it', 'sue',
       'avg_revision_next_6_months', 'avg_revision_from_month_7'],
      dtype='object')


...

Index(['permno', 'date', 'ret', 'gvkey', 'datadate', 'ni', 'mv', 'bv', 'cf',
       'rdq', 'market_ret', 'abr', 'abr_1', 'abr_2', 'abr_3', 'abr_4', 'abr_5',
       'abr_6', 'abr_7', 'abr_8', 'ticker', 'statpers', 'meanest', 'revision',
       'six_month_return', 'return_1y_after', 'return_2y_after',
       'return_3y_after', 'future_six_month_return', 'book_to_market',
       'cash_flow_to_price', 'earnings_diff', 'F_it', 'sue',
       'avg_revision_next_6_months', 'avg_revision_from_month_7'],
      dtype='object')


Processing:   0%|          | 15/20015 [00:11<5:05:27,  1.09chunk/s, remaining_time=4:22:41.643410]

Index(['permno', 'date', 'ret', 'gvkey', 'datadate', 'ni', 'mv', 'bv', 'cf',
       'rdq', 'market_ret', 'abr', 'abr_1', 'abr_2', 'abr_3', 'abr_4', 'abr_5',
       'abr_6', 'abr_7', 'abr_8', 'ticker', 'statpers', 'meanest', 'revision',
       'six_month_return', 'return_1y_after', 'return_2y_after',
       'return_3y_after', 'future_six_month_return', 'book_to_market',
       'cash_flow_to_price', 'earnings_diff', 'F_it', 'sue',
       'avg_revision_next_6_months', 'avg_revision_from_month_7'],
      dtype='object')


Processing:   0%|          | 15/20015 [00:12<4:37:10,  1.20chunk/s, remaining_time=4:22:41.643410]


KeyboardInterrupt: ignored

In [ ]:
print(pd.read_csv(final_data_path, nrows=5))

    permno        date       ret  gvkey    datadate     ni    mv      bv  \
0  10001.0  2000-01-31 -0.044118  12994  2000-06-30  1.297  19.8  13.961   
1  10001.0  2000-01-31 -0.044118  12994  2000-06-30  1.297  19.8  13.961   
2  10001.0  2000-01-31 -0.044118  12994  2000-06-30  1.297  19.8  13.961   
3  10001.0  2000-01-31 -0.044118  12994  2000-06-30  1.297  19.8  13.961   
4  10001.0  2000-01-31 -0.044118  12994  2000-06-30  1.297  19.8  13.961   

      cf         rdq  ...  statpers  meanest  revision  six_month_return  \
0  0.616  2000-02-15  ...       NaN      NaN       NaN         -0.044118   
1  0.616  2000-05-18  ...       NaN      NaN       NaN         -0.088235   
2  0.616  2000-09-28  ...       NaN      NaN       NaN         -0.132353   
3  0.616  2000-11-14  ...       NaN      NaN       NaN         -0.176471   
4  0.616  2001-02-14  ...       NaN      NaN       NaN         -0.220588   

   return_1y_after  return_2y_after  return_3y_after  future_six_month_return  \
0    

In [ ]:
import pandas as pd

# Load data
file_path = 'final_merged_data.csv'
final_data = pd.read_csv(file_path, error_bad_lines=False)
final_data['date'] = pd.to_datetime(final_data['date'])

# Ensure data is sorted by time and permno
final_data.sort_values(by=['permno', 'date'], inplace=True)

# Calculate future returns for each different time period
time_periods = [6, 12, 24, 36]  # 6 months, 1 year, 2 years, 3 years
for period in time_periods:
    col_name = f'future_{period}_month_return'
    final_data[col_name] = final_data.groupby('permno')['ret'].rolling(window=period).sum().shift(-period).reset_index(level=0, drop=True)

# Filter data: Only keep rows with necessary return data
final_data.dropna(subset=['six_month_return', 'book_to_market', 'cash_flow_to_price'] + [f'future_{period}_month_return' for period in time_periods], inplace=True)

# Get unique months
unique_dates = final_data['date'].unique()

# Initialize result DataFrame
result_data = pd.DataFrame()

# Classify stocks based on selected method
def classify_by_choice(monthly_data, choice):
    if choice == "1":
        return monthly_data.sort_values(by='six_month_return', ascending=False)
    elif choice == "2":
        return monthly_data.sort_values(by='sue', ascending=False)
    elif choice == "3":
        return monthly_data.sort_values(by='abr_1', ascending=False)  # Assuming using the most recent month's abnormal return
    elif choice == "4":
        return monthly_data.sort_values(by='revision', ascending=False)

# Rank and create decile groups for each month
for date in unique_dates:
    # Get data for the current month
    monthly_data = final_data[final_data['date'] == date].copy()

    # Check if monthly_data is empty, if so, continue to the next iteration
    if monthly_data.empty:
        continue

    # Rank stocks based on the selected method
    if choice != "5":
        monthly_data = classify_by_choice(monthly_data, choice)
    else:
        for ch in ["1", "2", "3", "4"]:
            monthly_data = classify_by_choice(monthly_data, ch)

    # Calculate the size of each decile
    decile_size = len(monthly_data) // 10

    # Assign a decile to each stock
    monthly_data['decile'] = (monthly_data.index // decile_size) + 1
    monthly_data.loc[monthly_data['decile'] > 10, 'decile'] = 10  # Handle potential size imbalance in the last decile

    # Append monthly data to the result DataFrame
    result_data = pd.concat([result_data, monthly_data], axis=0)

# Repeat the process for each month
for date in unique_dates:
    # Get data for the current month
    monthly_data = final_data[final_data['date'] == date].copy()

    # Rank stocks based on six-month returns
    monthly_data.sort_values(by='six_month_return', ascending=False, inplace=True)

    # Calculate the size of each decile
    decile_size = len(monthly_data) // 10

    # Assign a decile to each stock
    monthly_data['decile'] = (monthly_data.index // decile_size) + 1
    monthly_data.loc[monthly_data['decile'] > 10, 'decile'] = 10  # Handle potential size imbalance in the last decile

    # Append monthly data to the result DataFrame
    result_data = pd.concat([result_data, monthly_data], axis=0)

# Print Panel A and Panel B
columns = ['"1\n(Low)"'] + [str(i) for i in range(2, 10)] + ['"10\n(High)"']
print("Panel A: Returns")
for period in time_periods:
    col_name = f'future_{period}_month_return'
    print(f'Return {period} months after portfolio formation', end="\t")
    average_returns = result_data.groupby('decile')[col_name].mean()
    for col, ret in zip(columns, average_returns):
        print("{:.3f}".format(ret), end="\t")
    print()

print("\nPanel B: Characteristics")

# Calculate and print Book-to-market ratio for each decile
print("Book-to-market ratio", end="\t")
average_book_to_market = result_data.groupby('decile')['book_to_market'].mean()
for col, ratio in zip(columns, average_book_to_market):
    print("{:.3f}".format(ratio), end="\t")
print()

# Calculate and print Cash flow-to-price ratio for each decile
print("Cash flow-to-price ratio", end="\t")
average_cash_flow_to_price = result_data.groupby('decile')['cash_flow_to_price'].mean()
for col, ratio in zip(columns, average_cash_flow_to_price):
    print("{:.3f}".format(ratio), end="\t")
print()

# Print Panel C: Standardized Unexpected Earnings
print("\nPanel C: Standardized Unexpected Earnings")
quarters = ['Most recent quarter', 'Next quarter']  # Assuming data for these two quarters
for quarter in quarters:
    print(quarter, end="\t")
    # Here assuming the column 'sue' already contains the respective quarter's data
    average_earnings = result_data.groupby('decile')['sue'].mean()
    for col, earnings in zip(columns, average_earnings):
        print("{:.3f}".format(earnings), end="\t")
    print()

# Panel D: Abnormal Return Around Earnings Announcements
print("\nPanel D: Abnormal Return Around Earnings Announcements")

announcement_periods = [1, 2, 3, 4, 5]  # 1 month before earnings announcement, 1, 2, 3, 4 months after

for period in announcement_periods:
    col_name = f'abr_{period}'
    if period == 1:
        print(f"Most recent announcement 1 month before", end="\t")
    else:
        print(f"{period-1} announcement(s) after portfolio formation", end="\t")

    if col_name in result_data.columns:
        average_abr = result_data.groupby('decile')[col_name].mean()
        for decile, abr in zip(range(1, 11), average_abr):
            print(f"{abr:.3f}", end="\t")
    else:
        print(f"Warning: Column {col_name} not found in the data!")
    print()

# Panel E: Revision in Analyst Forecasts
print("\nPanel E: Revision in Analyst Forecasts")

# Calculate "Average over next 6 months"
result_data.reset_index(drop=True, inplace=True)  # Reset index
result_data['avg_revision_next_6_months'] = result_data.groupby('permno')['revision'].rolling(window=6).mean().reset_index(level=0, drop=True).shift(-6)

# Calculate "Average from months 7"
result_data['avg_revision_from_month_7'] = result_data.groupby('permno')['revision'].expanding(min_periods=7).mean().reset_index(level=0, drop=True).shift(-7)

forecast_periods = ["revision", "avg_revision_next_6_months", "avg_revision_from_month_7"]
forecast_period_names = ["Most recent revision", "Average over next 6 months", "Average from months 7"]

for period, period_name in zip(forecast_periods, forecast_period_names):
    col_name = period
    print(period_name, end="\t")

    if col_name in result_data.columns:
        average_revision = result_data.groupby('decile')[col_name].mean()
        for decile, revision in zip(range(1, 11), average_revision):
            print(f"{revision:.3f}", end="\t")
    else:
        print(f"Warning: Column {col_name} not found in the data!")
    print()


<ipython-input-28-268bad4ab60c>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  final_data = pd.read_csv(file_path, error_bad_lines=False)


Panel A: Returns
Return 6 months after portfolio formation	-0.808	-2.018	-2.612	-0.034	0.185	-0.806	0.773	-0.683	0.360	0.901	
Return 12 months after portfolio formation	-1.616	-4.036	-5.223	-0.067	0.371	-1.612	1.546	-1.366	0.720	1.802	
Return 24 months after portfolio formation	-3.232	-8.073	-10.447	-0.135	0.741	-3.224	3.091	-2.732	1.440	3.604	
Return 36 months after portfolio formation	-4.848	-12.109	-15.670	-0.202	1.112	-4.836	4.637	-4.099	2.160	5.406	

Panel B: Characteristics
Book-to-market ratio	0.389	3.323	3.544	0.015	0.434	0.300	1.724	0.461	0.320	0.630	
Cash flow-to-price ratio	0.085	0.203	-0.022	-0.025	0.037	0.056	0.345	0.055	-0.388	0.132	

Panel C: Standardized Unexpected Earnings
Most recent quarter	-0.034	0.022	-0.008	0.006	-0.004	-0.021	-0.012	0.042	-0.045	-0.018	
Next quarter	-0.034	0.022	-0.008	0.006	-0.004	-0.021	-0.012	0.042	-0.045	-0.018	

Panel D: Abnormal Return Around Earnings Announcements
Most recent announcement 1 month before	nan	nan	nan	nan	nan	nan	nan	0.389	na

In [ ]:
import pandas as pd

# Load data
file_path = 'final_merged_data.csv'
final_data = pd.read_csv(file_path, error_bad_lines=False)
final_data['date'] = pd.to_datetime(final_data['date'])

# Ensure data is sorted by time and permno
final_data.sort_values(by=['permno', 'date'], inplace=True)

# Calculate future returns for each different time period
time_periods = [6, 12, 24, 36]  # 6 months, 1 year, 2 years, 3 years
for period in time_periods:
    col_name = f'future_{period}_month_return'
    final_data[col_name] = final_data.groupby('permno')['ret'].rolling(window=period).sum().shift(-period).reset_index(level=0, drop=True)

# Filter data: Only keep rows with necessary return data
final_data.dropna(subset=['six_month_return', 'book_to_market', 'cash_flow_to_price'] + [f'future_{period}_month_return' for period in time_periods], inplace=True)

# Get unique months
unique_dates = final_data['date'].unique()

# Initialize result DataFrame
result_data = pd.DataFrame()

# Function to classify stocks based on selected method
def classify_by_choice(monthly_data, choice):
    if choice == "1":
        return monthly_data.sort_values(by='six_month_return', ascending=False)
    elif choice == "2":
        return monthly_data.sort_values(by='sue', ascending=False)
    elif choice == "3":
        return monthly_data.sort_values(by='abr_1', ascending=False)  # Assuming using the most recent month's abnormal return
    elif choice == "4":
        return monthly_data.sort_values(by='revision', ascending=False)

# Prompt user for choice of ranking method
print("Please select a ranking method:")
print("1. Six-month return")
print("2. Standardized Unexpected Earnings (SUE)")
print("3. Abnormal Return before earnings announcement (ABR)")
print("4. Analyst forecast revision")
print("5. All of the above methods")
choice = input("Enter your choice (1/2/3/4/5):")

# Initialize list for result tables
result_tables = []

# Generate tables for each choice
if choice == "5":
    for ch in ["1", "2", "3", "4"]:
        result_data = pd.DataFrame()
        for date in unique_dates:
            monthly_data = final_data[final_data['date'] == date].copy()
            if monthly_data.empty:
                continue
            monthly_data = classify_by_choice(monthly_data, ch)
            decile_size = len(monthly_data) // 10
            monthly_data['decile'] = (monthly_data.index // decile_size) + 1
            monthly_data.loc[monthly_data['decile'] > 10, 'decile'] = 10
            result_data = pd.concat([result_data, monthly_data], axis=0)
        result_tables.append(result_data)
else:
    result_data = pd.DataFrame()
    for date in unique_dates:
        monthly_data = final_data[final_data['date'] == date].copy()
        if monthly_data.empty:
            continue
        monthly_data = classify_by_choice(monthly_data, choice)
        decile_size = len(monthly_data) // 10
        monthly_data['decile'] = (monthly_data.index // decile_size) + 1
        monthly_data.loc[monthly_data['decile'] > 10, 'decile'] = 10
        result_data = pd.concat([result_data, monthly_data], axis=0)
    result_tables.append(result_data)

# Print Panel A and Panel B
columns = ['"1\n(Low)"'] + [str(i) for i in range(2, 10)] + ['"10\n(High)"']
print("Panel A: Returns")
for i, table in enumerate(result_tables):
    print(f"Table {i + 1}")
    for period in time_periods:
        col_name = f'future_{period}_month_return'
        print(f'Return {period} months after portfolio formation', end="\t")
        average_returns = table.groupby('decile')[col_name].mean()
        for col, ret in zip(columns, average_returns):
            print("{:.3f}".format(ret), end="\t")
        print()

    print("\nPanel B: Characteristics")

    # Calculate and print Book-to-market ratio for each decile
    print("Book-to-market ratio", end="\t")
    average_book_to_market = table.groupby('decile')['book_to_market'].mean()
    for col, ratio in zip(columns, average_book_to_market):
        print("{:.3f}".format(ratio), end="\t")
    print()

    # Calculate and print Cash flow-to-price ratio for each decile
    print("Cash flow-to-price ratio", end="\t")
    average_cash_flow_to_price = table.groupby('decile')['cash_flow_to_price'].mean()
    for col, ratio in zip(columns, average_cash_flow_to_price):
        print("{:.3f}".format(ratio), end="\t")
    print()

    # Print Panel C: Standardized Unexpected Earnings
    print("\nPanel C: Standardized Unexpected Earnings")
    for quarter in quarters:
        print(quarter, end="\t")
        average_earnings = table.groupby('decile')['sue'].mean()
        for col, earnings in zip(columns, average_earnings):
            print("{:.3f}".format(earnings), end="\t")
        print()

    # Panel D: Abnormal Return Around Earnings Announcements
    print("\nPanel D: Abnormal Return Around Earnings Announcements")

    for period in announcement_periods:
        col_name = f'abr_{period}'
        if period == 1:
            print(f"Most recent announcement 1 month before", end="\t")
        else:
            print(f"{period-1} announcement(s) after portfolio formation", end="\t")

        if col_name in table.columns:
            average_abr = table.groupby('decile')[col_name].mean()
            for decile, abr in zip(range(1, 11), average_abr):
                print(f"{abr:.3f}", end="\t")
        else:
            print(f"Warning: Column {col_name} not found in the data!")
        print()

    # Panel E: Revision in Analyst Forecasts
    print("\nPanel E: Revision in Analyst Forecasts")

    result_table.reset_index(drop=True, inplace=True)
    result_table['avg_revision_next_6_months'] = result_table.groupby('permno')['revision'].rolling(window=6).mean().reset_index(level=0, drop=True).shift(-6)
    result_table['avg_revision_from_month_7'] = result_table.groupby('permno')['revision'].expanding(min_periods=7).mean().reset_index(level=0, drop=True).shift(-7)

    forecast_periods = ["revision", "avg_revision_next_6_months", "avg_revision_from_month_7"]
    forecast_period_names = ["Most recent revision", "Average over next 6 months", "Average from months 7"]

    for period, period_name in zip(forecast_periods, forecast_period_names):
        col_name = period
        print(period_name, end="\t")

        if col_name in result_table.columns:
            average_revision = result_table.groupby('decile')[col_name].mean()
            for decile, revision in zip(range(1, 11), average_revision):
                print(f"{revision:.3f}", end="\t")
        else:
            print(f"Warning: Column {col_name} not found in the data!")
        print()


<ipython-input-26-2fe8b2424700>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  final_data = pd.read_csv(file_path, error_bad_lines=False)


请选择排名方式：
1. 六个月收益率
2. 标准化意外收益（SUE）
3. 财报发布前一个月的异常回报（ABR）
4. 分析师预测修订
5. 所有上述方式
请输入选择（1/2/3/4/5）：5
Panel A: Returns
Table 1
Return 6 months after portfolio formation	-0.845	-2.510	-1.782	0.152	-0.504	0.470	-0.376	0.090	3.004	0.498	
Return 12 months after portfolio formation	-1.689	-5.019	-3.564	0.304	-1.008	0.939	-0.753	0.180	6.008	0.996	
Return 24 months after portfolio formation	-3.378	-10.038	-7.128	0.608	-2.015	1.878	-1.506	0.360	12.016	1.992	
Return 36 months after portfolio formation	-5.067	-15.057	-10.692	0.912	-3.023	2.818	-2.259	0.540	18.025	2.988	

Panel B: Characteristics
Book-to-market ratio	0.367	4.283	2.147	-0.044	0.343	1.518	0.532	0.342	0.427	0.695	
Cash flow-to-price ratio	0.078	0.250	-0.128	-0.048	0.068	0.312	0.048	-0.266	0.018	0.143	

Panel C: Standardized Unexpected Earnings
Most recent quarter	-0.026	0.003	0.032	-0.055	0.003	-0.016	0.018	-0.029	-0.038	-0.008	
Next quarter	-0.026	0.003	0.032	-0.055	0.003	-0.016	0.018	-0.029	-0.038	-0.008	

Panel D: Abnormal Return Arou

NameError: ignored